In [87]:
import sagemaker
from sklearn.model_selection import train_test_split
import boto3 #to connect s3 bucket
import pandas as pd


region = 'us-east-1'
session = boto3.session.Session(region_name=region)
sm_boto3 = session.client('sagemaker')
sess = sagemaker.Session(boto_session=session)

bucket = "mobbucketsagemaker21"
print("Using bucket " + bucket)


Using bucket mobbucketsagemaker21


In [63]:
df=pd.read_csv("/Users/samarthraipal/Downloads/mobile_dataset/train.csv")

In [64]:
df.head()

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,...,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
0,842,0,2.2,0,1,0,7,0.6,188,2,...,20,756,2549,9,7,19,0,0,1,1
1,1021,1,0.5,1,0,1,53,0.7,136,3,...,905,1988,2631,17,3,7,1,1,0,2
2,563,1,0.5,1,2,1,41,0.9,145,5,...,1263,1716,2603,11,2,9,1,1,0,2
3,615,1,2.5,0,0,0,10,0.8,131,6,...,1216,1786,2769,16,8,11,1,0,0,2
4,1821,1,1.2,0,13,1,44,0.6,141,2,...,1208,1212,1411,8,2,15,1,1,0,1


In [65]:
df.shape

(2000, 21)

In [66]:
df["price_range"].value_counts(normalize=True)

price_range
1    0.25
2    0.25
3    0.25
0    0.25
Name: proportion, dtype: float64

In [67]:
df.columns

Index(['battery_power', 'blue', 'clock_speed', 'dual_sim', 'fc', 'four_g',
       'int_memory', 'm_dep', 'mobile_wt', 'n_cores', 'pc', 'px_height',
       'px_width', 'ram', 'sc_h', 'sc_w', 'talk_time', 'three_g',
       'touch_screen', 'wifi', 'price_range'],
      dtype='object')

In [68]:
df.isnull().mean()

battery_power    0.0
blue             0.0
clock_speed      0.0
dual_sim         0.0
fc               0.0
four_g           0.0
int_memory       0.0
m_dep            0.0
mobile_wt        0.0
n_cores          0.0
pc               0.0
px_height        0.0
px_width         0.0
ram              0.0
sc_h             0.0
sc_w             0.0
talk_time        0.0
three_g          0.0
touch_screen     0.0
wifi             0.0
price_range      0.0
dtype: float64

In [69]:
features=list(df.columns)
features

['battery_power',
 'blue',
 'clock_speed',
 'dual_sim',
 'fc',
 'four_g',
 'int_memory',
 'm_dep',
 'mobile_wt',
 'n_cores',
 'pc',
 'px_height',
 'px_width',
 'ram',
 'sc_h',
 'sc_w',
 'talk_time',
 'three_g',
 'touch_screen',
 'wifi',
 'price_range']

In [70]:
label=features.pop(-1)
label

'price_range'

In [71]:
x=df[features]
y=df[labels]

In [72]:
x.head()

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,pc,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi
0,842,0,2.2,0,1,0,7,0.6,188,2,2,20,756,2549,9,7,19,0,0,1
1,1021,1,0.5,1,0,1,53,0.7,136,3,6,905,1988,2631,17,3,7,1,1,0
2,563,1,0.5,1,2,1,41,0.9,145,5,6,1263,1716,2603,11,2,9,1,1,0
3,615,1,2.5,0,0,0,10,0.8,131,6,9,1216,1786,2769,16,8,11,1,0,0
4,1821,1,1.2,0,13,1,44,0.6,141,2,14,1208,1212,1411,8,2,15,1,1,0


In [73]:
y.head()

0    1
1    2
2    2
3    2
4    1
Name: price_range, dtype: int64

In [74]:
x.shape

(2000, 20)

In [75]:
X_train,X_test,y_train,y_test=train_test_split(x,y,test_size=0.15,random_state=0)

In [76]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)


(1700, 20)
(300, 20)
(1700,)
(300,)


In [77]:
trainX=pd.DataFrame(X_train)
 
trainX["label"] = y_train
 



testX=pd.DataFrame(X_test)
testX["label"]=y_test

In [78]:
print(trainX.shape)


(1700, 21)


In [79]:
trainX.isnull().sum()
testX.isnull().sum()

battery_power    0
blue             0
clock_speed      0
dual_sim         0
fc               0
four_g           0
int_memory       0
m_dep            0
mobile_wt        0
n_cores          0
pc               0
px_height        0
px_width         0
ram              0
sc_h             0
sc_w             0
talk_time        0
three_g          0
touch_screen     0
wifi             0
label            0
dtype: int64

In [80]:
trainX.to_csv("train-V-1.csv",index=False)
trainX.to_csv("test-V-1.csv",index=False)

In [81]:
sk_prefix="sagemaker/mpbile_price_classification/sklearncontainer" #folder 

trainpath=sess.upload_data(path="train-V-1.csv",bucket=bucket,key_prefix=sk_prefix)

testpath=sess.upload_data(path="test-V-1.csv",bucket=bucket,key_prefix=sk_prefix)

In [82]:
print(trainpath)

s3://mobbucketsagemaker21/sagemaker/mpbile_price_classification/sklearncontainer/train-V-1.csv


In [94]:
%%writefile script.py

import argparse
import os
import json
import sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, precision_score, recall_score
import joblib
import pathlib
from io import StringIO
import boto3
import pandas as pd
import numpy as np

def model_fn(model_dir):
    clf = joblib.load(os.path.join(model_dir, "model.joblib"))
    return clf

if __name__ =='__main__':

    print("[INFO] Extracting arguments")
    parser = argparse.ArgumentParser()

    # hyperparameters sent by the client are passed as command-line arguments to the script.
    parser.add_argument('--n_estimators', type=int, default=100)
    parser.add_argument('--random_state', type=int, default=0)
    # parser.add_argument('--epochs', type=int, default=10)
    # parser.add_argument('--batch-size', type=int, default=100)
    # parser.add_argument('--learning-rate', type=float, default=0.1)

    # an alternative way to load hyperparameters via SM_HPS environment variable.
    # parser.add_argument('--sm-hps', type=json.loads, default=os.environ['SM_HPS'])

    # input data and model directories
    parser.add_argument('--model-dir', type=str, default=os.environ['SM_MODEL_DIR'])
    parser.add_argument('--train', type=str, default=os.environ['SM_CHANNEL_TRAIN'])
    parser.add_argument('--test', type=str, default=os.environ['SM_CHANNEL_TEST'])
    parser.add_argument('--train_file', type=str, default='train-V-1.csv')
    parser.add_argument('--test_file', type=str, default='test-V-1.csv')

    args, _ = parser.parse_known_args()

    print("SKLearn Version: ", sklearn.__version__)
    print("Joblib Version: ", joblib.__version__)

    print("[INFO] Reading data")
    print()
    train_df = pd.read_csv(os.path.join(args.train, args.train_file))
    test_df = pd.read_csv(os.path.join(args.test, args.test_file))

    features = list(train_df.columns)
    label = features.pop(-1)
    
    print("Building training and testing datasets")
    print()
    X_train = train_df[features]
    X_test = test_df[features]
    y_train = train_df[label]
    y_test = test_df[label]

    print('Column order: ')
    print(features)
    print()
    
    print("Label column is: ",label)
    print()
    
    print("Data Shape: ")
    print()
    print("---- SHAPE OF TRAINING DATA (80%) ----")
    print(X_train.shape)
    print(y_train.shape)
    print()
    print("---- SHAPE OF TESTING DATA (20%) ----")
    print(X_test.shape)
    print(y_test.shape)
    print()
    
    print("Training RandomForest Model.....")
    print()
    model = RandomForestClassifier(n_estimators=args.n_estimators, random_state=args.random_state, verbose=3, n_jobs=None)
    model.fit(X_train, y_train)
    print()

    model_path = os.path.join(args.model_dir, "model.joblib")
    joblib.dump(model, model_path)
    print("Model persisted at " + model_path)
    print()

    
    y_pred_test = model.predict(X_test)
    test_acc = accuracy_score(y_test,y_pred_test)
    test_rep = classification_report(y_test,y_pred_test)

    print()
    print("---- METRICS RESULTS FOR TESTING DATA ----")
    print()
    print("Total Rows are: ", X_test.shape[0])
    print('[TESTING] Model Accuracy is: ', test_acc)
    print('[TESTING] Testing Report: ')
    print(test_rep)

Overwriting script.py


In [95]:

from sagemaker.sklearn.estimator import SKLearn

FRAMEWORK_VERSION = "0.23-1"

sklearn_estimator = SKLearn(
    entry_point = "script.py",
    role="arn:aws:iam::637423611339:role/service-role/AmazonSageMaker-ExecutionRole-20240621T005369",
    instance_count=1,
    instance_type="ml.m5.large",
    framework_version=FRAMEWORK_VERSION,
    base_job_name="RF-custom-sklearn",
    hyperparameters={
        "n_estimators": 100,
        "random_state": 0,
    },
    use_spot_instances = True,
    max_wait = 7200,
    max_run = 3600
)

In [96]:
sklearn_estimator.fit({"train":trainpath,"test":testpath},wait=True)

INFO:sagemaker:Creating training-job with name: RF-custom-sklearn-2024-08-06-23-07-45-954


2024-08-06 23:07:48 Starting - Starting the training job...
2024-08-06 23:08:01 Starting - Preparing the instances for training...
2024-08-06 23:08:33 Downloading - Downloading input data...
2024-08-06 23:08:59 Downloading - Downloading the training image...
2024-08-06 23:09:45 Training - Training image download completed. Training in progress..2024-08-06 23:09:49,715 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2024-08-06 23:09:49,718 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-08-06 23:09:49,761 sagemaker_sklearn_container.training INFO     Invoking user training script.
2024-08-06 23:09:49,927 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-08-06 23:09:49,940 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-08-06 23:09:49,952 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-08-06 23:

In [98]:
sklearn_estimator.latest_training_job.wait(logs="None")
artifact=sm_boto3.describe_training_job(TrainingJobName=sklearn_estimator.latest_training_job.name)["ModelArtifacts"]["S3ModelArtifacts"]

print("Model artifcats persisted at "+ artifact)


2024-08-06 23:10:13 Starting - Preparing the instances for training
2024-08-06 23:10:13 Downloading - Downloading the training image
2024-08-06 23:10:13 Training - Training image download completed. Training in progress.
2024-08-06 23:10:13 Uploading - Uploading generated training model
2024-08-06 23:10:13 Completed - Training job completed
Model artifcats persisted at s3://sagemaker-us-east-1-637423611339/RF-custom-sklearn-2024-08-06-23-07-45-954/output/model.tar.gz


In [110]:
from sagemaker.sklearn.model import SKLearnModel
from time import strftime,gmtime


model_name= "Custom-sklearn-model" + strftime("%Y-%m-%d-%H-%M-%S", gmtime()).replace(':', '-').replace('/', '-')

model=SKLearnModel(name=model_name,model_data=artifact,role="arn:aws:iam::637423611339:role/service-role/AmazonSageMaker-ExecutionRole-20240621T005369",
                   entry_point="script.py",
                   framework_version=FRAMEWORK_VERSION)

In [111]:
model

In [112]:
endpoint_name = "Custom-sklearn-model" + strftime("%Y-%m-%d-%H-%M-%S", gmtime()).replace(':', '-').replace('/', '-')

print("EndpointName={}".format(endpoint_name))

predictor=model.deploy(initial_instance_count=1,instance_type="ml.m4.xlarge",endpoint_name=endpoint_name,)


EndpointName=Custom-sklearn-model2024-08-07-03-00-51


INFO:sagemaker:Creating model with name: Custom-sklearn-model2024-08-07-03-00-47
INFO:sagemaker:Creating endpoint-config with name Custom-sklearn-model2024-08-07-03-00-51
INFO:sagemaker:Creating endpoint with name Custom-sklearn-model2024-08-07-03-00-51


------!

In [113]:
endpoint_name

'Custom-sklearn-model2024-08-07-03-00-51'

In [116]:
testX[features][0:2].values.tolist() #first 2 records

[[1454.0,
  1.0,
  0.5,
  1.0,
  1.0,
  0.0,
  34.0,
  0.7,
  83.0,
  4.0,
  3.0,
  250.0,
  1033.0,
  3419.0,
  7.0,
  5.0,
  5.0,
  1.0,
  1.0,
  0.0],
 [1092.0,
  1.0,
  0.5,
  1.0,
  10.0,
  0.0,
  11.0,
  0.5,
  167.0,
  3.0,
  14.0,
  468.0,
  571.0,
  737.0,
  14.0,
  4.0,
  11.0,
  0.0,
  1.0,
  0.0]]

In [117]:
print(predictor.predict(testX[features][0:2].values.tolist()))

[3 0]


sm_boto3.delete_endpoint(EndpointName=endpoint_name)